In [73]:
# Installing necessary packages
!pip install openpyxl

# Importing packages
import pandas as pd
import numpy as np
from datetime import datetime
import visualization
import math

You should consider upgrading via the 'C:\Users\emmet\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [74]:
'''
OpenFEMA Dataset: FIMA NFIP Redacted Claims - Version 2
Last Data Refresh: 04-16-2024
Documentation: https://www.fema.gov/openfema-data-page/fima-nfip-redacted-claims-v2
'''
fema_redacted_claims_raw = pd.read_csv("FimaNfipClaims.csv")
fema_redacted_claims_raw.head(5)

C:\Users\emmet\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3550: DtypeWarning: Columns (7,35,38,42,51,52,61) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,agricultureStructureIndicator,asOfDate,basementEnclosureCrawlspaceType,policyCount,crsClassificationCode,dateOfLoss,elevatedBuildingIndicator,elevationCertificateIndicator,elevationDifference,baseFloodElevation,...,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-03-27T12:15:45.887Z,0.0,1,NaN,2014-09-08T00:00:00.000Z,0,2,NaN,NaN,...,0,AZ,Currently Unavailable,85015.0,4013.0,4.013109e+09,4.013109e+10,33.5,-112.1,aa9ad641-4580-4d66-ac05-d51fc3926feb
1,0,2022-09-11T16:35:19.755Z,0.0,1,NaN,2012-10-29T00:00:00.000Z,0,NaN,NaN,NaN,...,0,NJ,Currently Unavailable,8740.0,34029.0,3.402973e+10,3.402973e+11,39.9,-74.1,53fa451b-953b-4ec2-8d93-311a033d75a8
2,0,2020-01-22T16:55:53.194Z,NaN,1,NaN,2017-09-10T00:00:00.000Z,1,NaN,NaN,NaN,...,0,FL,Currently Unavailable,32207.0,12031.0,1.203100e+10,1.203100e+11,30.3,-81.7,5cf657e2-d78e-4c99-9dc3-a585efc12e13
3,0,2020-01-22T16:55:53.194Z,NaN,1,NaN,2012-08-29T00:00:00.000Z,0,NaN,NaN,NaN,...,0,LA,Currently Unavailable,70068.0,22095.0,2.209507e+10,2.209507e+11,30.1,-90.5,2f77a314-d3af-44ff-90f0-03c639fbbb5a
4,0,2020-01-22T16:55:53.194Z,NaN,1,NaN,2016-03-09T00:00:00.000Z,0,NaN,NaN,NaN,...,0,LA,Currently Unavailable,71203.0,22073.0,2.207301e+10,2.207301e+11,32.6,-92.1,32fcc4a1-28d5-46ae-8c41-86c2b66eda05


In [75]:
'''
Augmenting dataset with EM-DAT Public Table
Documentation: https://doc.emdat.be/docs/data-structure-and-content/emdat-public-table/
Date of access: 4/29/2024
'''

em_dat_raw = pd.read_excel("public_emdat_custom_request.xlsx")
em_dat_raw.head(5)

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,Reconstruction Costs ('000 US$),"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update
0,1976-9152-BFA,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,BFA,...,NaN,NaN,NaN,NaN,NaN,NaN,19.891037,NaN,2006-11-24,2023-09-25
1,1976-9152-GMB,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,GMB,...,NaN,NaN,NaN,NaN,NaN,NaN,19.891037,NaN,2006-11-24,2023-09-25
2,1976-9152-MLI,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,MLI,...,NaN,NaN,NaN,NaN,NaN,NaN,21.408917,NaN,2006-11-24,2023-09-25
3,1976-9152-SEN,Yes,nat-cli-dro-dro,Natural,Climatological,Drought,Drought,NaN,NaN,SEN,...,NaN,NaN,NaN,NaN,300000.0,1508217.0,19.891037,NaN,2006-11-24,2023-09-25
4,1977-0001-ECU,Yes,nat-bio-epi-bac,Natural,Biological,Epidemic,Bacterial disease,NaN,Typhoid,ECU,...,NaN,NaN,NaN,NaN,NaN,NaN,19.891037,NaN,2003-07-01,2023-09-25


In [76]:
# Only interested in flooding events in the United States
flood_categories = ["Flood", "Storm", "Water", "Mass movement (wet)", "Glacial lake outburst flood"]
em_dat_df = em_dat_raw.loc[em_dat_raw["Disaster Type"].isin(flood_categories)].copy()
em_dat_df = em_dat_df.loc[em_dat_raw["ISO"] == "USA"]
# Create a datetime object for the start of each event
em_dat_df["Start Year"] = em_dat_df["Start Year"].fillna(1)
em_dat_df["Start Month"] = em_dat_df["Start Month"].fillna(1)
em_dat_df["Start Day"] = em_dat_df["Start Day"].fillna(1)
em_dat_df["Start Datetime"] = pd.to_datetime({
    'year': em_dat_df["Start Year"],
    'month': em_dat_df["Start Month"],
    'day': em_dat_df["Start Day"],
}).dt.tz_localize(None)
em_dat_df.head()

,DisNo.,Historic,Classification Key,Disaster Group,Disaster Subgroup,Disaster Type,Disaster Subtype,External IDs,Event Name,ISO,...,"Reconstruction Costs, Adjusted ('000 US$)",Insured Damage ('000 US$),"Insured Damage, Adjusted ('000 US$)",Total Damage ('000 US$),"Total Damage, Adjusted ('000 US$)",CPI,Admin Units,Entry Date,Last Update,Start Datetime
63,1977-0089-USA,Yes,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,NaN,NaN,USA,...,NaN,NaN,NaN,200000.0,1005478.0,19.891037,NaN,2003-07-01,2023-09-25,1977-07-22
88,1977-0118-USA,Yes,nat-hyd-flo-fla,Natural,Hydrological,Flood,Flash flood,NaN,NaN,USA,...,NaN,NaN,NaN,NaN,NaN,19.891037,NaN,2003-07-01,2023-09-25,1977-09-12
140,1977-0233-USA,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,NaN,USA,...,NaN,NaN,NaN,NaN,NaN,19.891037,NaN,2003-07-01,2023-09-25,1977-01-01
142,1977-0235-USA,Yes,nat-met-sto-sto,Natural,Meteorological,Storm,Storm (General),NaN,NaN,USA,...,NaN,NaN,NaN,NaN,NaN,19.891037,NaN,2003-07-01,2023-09-25,1977-04-01
146,1977-0245-USA,Yes,nat-hyd-flo-flo,Natural,Hydrological,Flood,Flood (General),NaN,NaN,USA,...,NaN,NaN,NaN,NaN,NaN,19.891037,NaN,2003-07-01,2023-09-25,1977-11-01


In [77]:
''' 
Input: A date and a string describing a natural disaster's name
Output: The magnitude and magnitude scale from the EM-DAT public table
Documentation of magnitudes: https://doc.emdat.be/docs/data-structure-and-content/hazard-and-disaster-magnitude-units/
'''
def event_magnitude(date_of_loss, event_name):
    # Find an event in the correct month in the EM-DAT database
    date_mask = (em_dat_df["Start Datetime"] >= date_of_loss - pd.DateOffset(months=1)) & (em_dat_df["Start Datetime"] <= date_of_loss + pd.DateOffset(days=2))
    potential_events = em_dat_df[date_mask].copy()
    potential_events["Time Difference"] = (potential_events["Start Datetime"] - date_of_loss).abs().dt.days
    if potential_events.shape[0] == 0:
        return np.nan, np.nan
    if potential_events.shape[0] == 1:
        return potential_events["Magnitude"].iloc[0], potential_events["Magnitude Scale"].iloc[0]
    potential_events = potential_events.sort_values(by="Time Difference", ascending=True)
    # If multiply flooding events happened in the US during a given month, check for one that matches the event name
    potential_events["Event Name"] = potential_events["Event Name"].str.replace('[^\w]', '', regex=True)
    event_name_mask = potential_events["Event Name"].apply(lambda x: isinstance(x, str) and isinstance(event_name, str) and (x.lower() in event_name.lower() or event_name.lower() in x.lower()))
    potential_events_names = potential_events[event_name_mask]
    if potential_events_names.shape[0] == 0:
        return potential_events["Magnitude"].iloc[0], potential_events["Magnitude Scale"].iloc[0]
    else:
        return potential_events_names["Magnitude"].iloc[0], potential_events_names["Magnitude Scale"].iloc[0]

# Example usage: Hurricane Katrina landfall
katrina_landfall = datetime(2005, 8, 29)
print("Hurricane Katrina Magnitude:", event_magnitude(katrina_landfall, "Katrina"))

Hurricane Katrina Magnitude: (280.0, 'Kph')


In [78]:
'''
Creating a label: total amount paid to policy holder for damage to the building
itself, the contents of the building, and increased cost of policy compliance.
'''

# Creating the label column
insurance_payments = fema_redacted_claims_raw["netBuildingPaymentAmount"] \
    + fema_redacted_claims_raw["netContentsPaymentAmount"] \
    + fema_redacted_claims_raw["netIccPaymentAmount"] 

# Dropping original components of labels
df = fema_redacted_claims_raw.drop(
    ["netBuildingPaymentAmount", 
    "netContentsPaymentAmount", 
    "netIccPaymentAmount"], axis=1)

df["totalAmountPaidOnClaim"] = insurance_payments

In [79]:
# Creating dummies for categorical features
df = pd.get_dummies(df, columns=["crsClassificationCode", 
                                 "elevationCertificateIndicator",
                                 "ratedFloodZone", "occupancyType", 
                                 "condominiumCoverageTypeCode", 
                                 "disasterAssistanceCoverageRequired", 
                                 "floodZoneCurrent"])

# Get the age of the building and the flood policy
current_year = pd.to_datetime("today").tz_localize(None).year
df["originalConstructionDate"] = df["originalConstructionDate"].fillna("1950").str[:4].astype(int)
df["yearsSinceConstruction"] = (current_year - df["originalConstructionDate"])
df["originalNBDate"] = df["originalNBDate"].fillna("1950").str[:4].astype(int)
df["yearsSinceNBPolicy"] = (current_year - df["originalNBDate"])
df = df.drop(["originalConstructionDate", "originalNBDate"], axis=1)

'''
Zip codes and census tracts which are numerically similar are also similar 
geographically, so we include these features. We also have latitude and 
longitude for each building for more granular information.
'''
df["reportedZipCode"] = df["reportedZipCode"].fillna(-1).astype(int)
df["censusTract"] = df["censusTract"].fillna(-1).astype(int)

''' 
Mappings for building and contents deductibles
Exact values taken from FEMA documentation
'''
deductible_mapping =  {"0" : 500, "1" : 1000, "2" : 2000, "3" : 3000, 
                       "4" : 4000, "5" : 5000, "9" : 750, "A" : 10000, 
                       "B" : 15000, "C" : 20000, "D" : 25000, "E" : 50000,
                       "F" : 1250, "G" : 1500, "H" : 200, "NULL" : 0}
df["buildingDeductibleCode"] = df["buildingDeductibleCode"].fillna("NULL").map(deductible_mapping)
df["contentsDeductibleCode"] = df["contentsDeductibleCode"].fillna("NULL").map(deductible_mapping)

In [ ]:
# Match claims to storm magnitudes and write result to csv (long computation time)
df["dateOfLoss"] = pd.to_datetime(df["dateOfLoss"]).dt.tz_localize(None)
df["floodEvent"] = df["floodEvent"].str.replace("[^\w]", "", regex=True)
magnitudes = df.apply(lambda row: event_magnitude(row["dateOfLoss"], row["floodEvent"]), axis=1)
df["Magnitude"] = magnitudes.apply(lambda x: x[0])
df["Magnitude Scale"] = magnitudes.apply(lambda x: x[1])
df.to_csv("matched_claims.csv")

In [82]:
# Select insurance claims which were matched to a significant storm
# Note: re-loading data because previous step takes >3hrs to compute
df_storms = pd.read_csv("matched_claims.csv")
df_storms = df_storms.dropna(subset=["Magnitude"], axis=0).copy()

C:\Users\emmet\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3550: DtypeWarning: Columns (8,33,39,47,48,57) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [83]:
'''
We note from the data that elevation information is missing regarding most of 
the properties. I query the US Geological Survey API to obtain approximate
topological information for each property.
'''

'''
Write longitude and longitude pairs to files of 500 pairs each, as USGS API only
allows for requests of 500 points at a time.
'''
longitude_latitude = df_storms[["longitude", "latitude"]].drop_duplicates()
num_coordinate_files = math.ceil(longitude_latitude.shape[0]/500.0)
coordinate_filenames = [f"coordinates/coordinates{i}.csv" for i in range(num_coordinate_files)]
for i in range(num_coordinate_files):
    upper = min(500*(i+1), longitude_latitude.shape[0])
    longitude_latitude.iloc[500*i:upper].to_csv(coordinate_filenames[i], index=False)

'''
Read all results from the USGS API and combine them into a single data frame
'''
coordinate_results_filenames = [f"coordinates/coordinates_results{i}.csv" for i in range(num_coordinate_files)]
usgs_dfs = [pd.read_csv(f).drop("ID", axis=1) for f in coordinate_results_filenames]
elevation_df = pd.concat(usgs_dfs, ignore_index=True)
elevation_df.set_index(["Input Lon", "Input Lat"], inplace=True)
elevation_df.head()

Elev(ft)  Elev(m)
Input Lon Input Lat                   
-81.7     30.3          21.33     6.50
-78.4     40.5        1226.12   373.72
-97.0     28.0          -0.36    -0.11
          28.1          -0.36    -0.11
-97.1     28.1          -0.36    -0.11

In [84]:
'''
Look up the elevation in feet for every property which is insured by FEMA
'''
df_storms = df_storms.dropna(subset=["longitude", "latitude"], axis=0)
df_storms.loc[:, "USGS_Elevation"] = [elevation_df.loc[(long, lat), "Elev(ft)"] for long, lat in zip(df_storms['longitude'], df_storms['latitude'])]

'''
Dropping metadata features used for matching and/or replacing with meaningful
features.
'''
def get_season(month):
    if month in [12, 1, 2]:
        return "winter"
    elif month in [3, 4, 5]:
        return "spring"
    elif month in [6, 7, 8]:
        return "summer"
    elif month in [9, 10, 11]:
        return "fall"

df_storms["dateOfLoss"] = pd.to_datetime(df_storms["dateOfLoss"]).dt.tz_localize(None)
df_storms["seasonOfLoss"] = df_storms["dateOfLoss"].dt.month.apply(get_season)
df_storms = pd.get_dummies(df_storms, columns=["seasonOfLoss"])
# Drop metadata features / redundant features / columns with measurement error
df_storms = df_storms.drop(["dateOfLoss", "floodEvent", 
                            "eventDesignationNumber", "censusBlockGroupFips", 
                            "id", "asOfDate", "rateMethod", "causeOfDamage", 
                            "nfipCommunityName", "nonPaymentReasonBuilding", 
                            "nonPaymentReasonContents", 
                            "condominiumCoverageTypeCode",
                            "nfipCommunityNumberCurrent", "floodZoneCurrent",
                            "reportedCity", "countyCode"], axis=1)
# Remove features related to those used for labeling (only known after claim paid)
df_storms = df_storms.drop(["amountPaidOnBuildingClaim",
                            "amountPaidOnContentsClaim",
                            "amountPaidOnIncreasedCostOfComplianceClaim"], 
                            axis=1)

In [85]:
'''
Transformation of remaining categorical features, filling in NaN values
'''

top_zones = df_storms['ratedFloodZone'].value_counts().head(10).index
df_storms['top_ratedFloodZone'] = df_storms['ratedFloodZone'].apply(lambda x: x if x in top_zones else 'Other')
flood_zone_dummies = pd.get_dummies(df_storms['top_ratedFloodZone'])
df_storms = pd.concat([df_storms, flood_zone_dummies], axis=1)
df_storms = df_storms.drop(["ratedFloodZone", "top_ratedFloodZone"], axis=1)

elev_cert_map = {'A': 5, 'B': 6, 'C': 7, 'D': 8, 'E': 9}
df_storms["elevationCertificateIndicator"] = df_storms["elevationCertificateIndicator"].map(lambda x: elev_cert_map.get(x, x))

df_storms = pd.get_dummies(df_storms, columns=["replacementCostBasis"])

df_storms["Magnitude Scale"] = np.where(df_storms["Magnitude Scale"] == "Km2", 1, 0)

# Fill remaining nan values with column means
df_storms.fillna(df_storms.mean(), inplace=True)

C:\Users\emmet\AppData\Local\Temp\ipykernel_24924\487773166.py:19: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_storms.fillna(df_storms.mean(), inplace=True)


In [86]:
'''
For the purposes of this project, I prepare datasets based on significant storms
in Florida and South Carolina. A significant storm is deemed one that is at
least a category 1 storm.
'''

# Filter for Florida and South Carolina
df_storms_FL = df_storms[df_storms["state"] == "FL"]
df_storms_SC = df_storms[df_storms["state"] == "SC"]
df_storms_FL = df_storms_FL.drop(["state"], axis=1)
df_storms_SC = df_storms_SC.drop(["state"], axis=1)

# Select only category 1 or greater tropical storms/depressions
df_storms_FL = df_storms_FL[(df_storms_FL["Magnitude"] > 75) & (df_storms_FL["Magnitude Scale"] == 0)]
df_storms_SC = df_storms_SC[(df_storms_SC["Magnitude"] > 75) & (df_storms_SC["Magnitude Scale"] == 0)]
df_storms_FL = df_storms_FL.drop(["Magnitude Scale"], axis=1)
df_storms_SC = df_storms_SC.drop(["Magnitude Scale"], axis=1)

# Serialize datasets
df_storms_FL.to_csv("StormClaimsFlorida.csv", index=False)
df_storms_SC.to_csv("StormClaimsSouthCarolina.csv", index=False)